In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from PIL import Image
import time
from tqdm.notebook import tqdm

In [2]:
# Enter display name and tag
display_name = 'your name'
player_tag = 'TAG' # No Hashtag

In [3]:
matches_url = f'https://api.henrikdev.xyz/valorant/v1/lifetime/matches/eu/{display_name}/{player_tag}'
maps_url = 'https://valorant-api.com/v1/maps'

In [4]:
maps_dict = {}

maps_resp = requests.get(maps_url)
maps = maps_resp.json()['data']

for m in maps:
    name = m['displayName']
    maps_dict[name] = {}
    maps_dict[name]['callouts'] = m['callouts']
    maps_dict[name]['map_url'] = m['displayIcon']
    maps_dict[name]['xMul'] = m['xMultiplier']
    maps_dict[name]['yMul'] = m['yMultiplier']
    maps_dict[name]['xAdd'] = m['xScalarToAdd']
    maps_dict[name]['yAdd'] = m['yScalarToAdd']
    maps_dict[name]['kills'] = []
    maps_dict[name]['deaths'] = []


In [5]:
matches_resp = requests.get(matches_url)
for i in tqdm(range(29)):
    match = matches_resp.json()['data'][i]
    kills = []
    deaths = []
    map_name = match['meta']['map']['name']
    match_id = match['meta']['id']
    match_url = f'https://api.henrikdev.xyz/valorant/v2/match/{match_id}'
    match_resp = requests.get(match_url)
    match_data = match_resp.json()['data']
    player_display_name = f'{display_name}#{player_tag}'
    for kill in match_data['kills']:
        killer_display_name = kill['killer_display_name']
        victim_display_name = kill['victim_display_name']
        if killer_display_name == player_display_name:
            for player in kill['player_locations_on_kill']:
                if player['player_display_name'] == player_display_name:
                    loc = np.array(list(player['location'].values()))[::-1]
                    ans = (np.array([maps_dict[map_name]['xMul'], maps_dict[map_name]['yMul']]) * loc 
                        + np.array([maps_dict[map_name]['xAdd'], maps_dict[map_name]['yAdd']])) * 1024
                    kills.append(tuple(ans))
        elif victim_display_name == player_display_name:
            loc = np.array(list(kill['victim_death_location'].values()))[::-1]
            ans = (np.array([maps_dict[map_name]['xMul'], maps_dict[map_name]['yMul']]) * loc 
                   + np.array([maps_dict[map_name]['xAdd'], maps_dict[map_name]['yAdd']])) * 1000
            deaths.append(tuple(ans))

    maps_dict[map_name]['kills'].extend(kills)
    maps_dict[map_name]['deaths'].extend(deaths)

    
            
for m in list(maps_dict.keys()):
    maps_dict[m]['kills'] = np.array(maps_dict[m]['kills'])
    maps_dict[m]['deaths'] = np.array(maps_dict[m]['deaths'])
    

  0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
for m in tqdm(list(maps_dict.keys())):    
    url = maps_dict[m]['map_url']
    try:
        kx, ky = maps_dict[m]['kills'].T
        dx, dy = maps_dict[m]['deaths'].T
    except:
        kx, ky, dx, dy = 0, 0, 0, 0
    
    if url:
        im = Image.open(requests.get(url, stream=True).raw)
        x = np.asarray(im.convert('RGBA')).copy()
        performance = np.zeros([x.shape[0], x.shape[1]])
        x_performance = np.empty((x.shape[0], x.shape[1], 4), dtype=np.uint8)
        frequency = np.zeros([x.shape[0], x.shape[1]])
        x_frequency = np.empty((x.shape[0], x.shape[1], 4), dtype=np.uint8)

        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                c_red = x[i, j, 0]
                c_green = x[i, j, 1]
                c_blue = x[i, j, 2]
                c_alpha = x[i, j, 3]
                if c_red >= 220 and c_green >= 220 and c_blue >= 220: 
                    x_performance[i, j] = (255, 255, 255, 255)
                    x_frequency[i, j] = (255, 255, 255, 255)
                elif 125<c_red<130 and 125<c_green<130 and 125<c_blue<130:
                    k_zones = np.sum(np.exp(-0.001*(kx-j)**2-0.001*(ky-i)**2))
                    d_zones = np.sum(np.exp(-0.001*(dx-j)**2-0.001*(dy-i)**2))
                    diff = (k_zones - d_zones)
                    summ = (k_zones + d_zones)
                    performance[i, j] = np.divide(diff, summ, where=summ!=0)
                    frequency[i, j] = summ
                    
                else:
                    x_performance[i, j] = (c_red, c_green, c_blue, c_alpha)
                    x_frequency[i, j] = (c_red, c_green, c_blue, c_alpha)
        
        frequency = np.divide(frequency, np.max(frequency), where=np.max(frequency)!=0)
        performance = np.divide(performance, np.max(performance), where=np.max(performance)!=0)
        performance *= np.power(frequency, 1/3)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                c_red = x[i, j, 0]
                c_green = x[i, j, 1]
                c_blue = x[i, j, 2]
                c_alpha = x[i, j, 3]
                if 125<c_red<130 and 125<c_green<130 and 125<c_blue<130:
                    freq_color = (int(255*frequency[i, j]), 0, int(255*(1-frequency[i, j])), 255)
                    x_frequency[i, j] = freq_color
                    perf_color = (int(127*(1-performance[i, j])), int(127*(1+performance[i, j])), 127, 255)
                    x_performance[i, j] = perf_color
                
        fig, ax = plt.subplots()
        
        im = Image.fromarray(x_performance.astype('uint8'), 'RGBA')      
        plt.imshow(im)
        plt.title(f'{m} performance')
                                  
        x1 = [k[0] for k in maps_dict[m]['kills'] if k[0] < 1000 and k[1] < 1000]
        y1 = [k[1] for k in maps_dict[m]['kills'] if k[0] < 1000 and k[1] < 1000]
        x2 = [d[0] for d in maps_dict[m]['deaths'] if d[0] < 1000 and d[1] < 1000]
        y2 = [d[1] for d in maps_dict[m]['deaths'] if d[0] < 1000 and d[1] < 1000]
        
        ax.scatter(x1, y1, s=10, c='b', marker='.')
        ax.scatter(x2, y2, s=10, c='k', marker='x')
        
        fig2, ax2 = plt.subplots()
                                  
        im2 = Image.fromarray(x_frequency.astype('uint8'), 'RGBA')      
        plt.imshow(im2)
        plt.title(f'{m} frequency')
                
        

  0%|          | 0/15 [00:00<?, ?it/s]

C:\Users\sfchi\AppData\Local\Temp\ipykernel_15968\3086568078.py:53: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
